In [12]:
# Import Library
import os
import json
import logging
from dotenv import load_dotenv
from datetime import datetime

# import pandas as pd
# import tiktoken
from langchain.chains import RetrievalQA, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import DataFrameLoader
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

In [4]:
# Load environment variables from .env
load_dotenv()

# Access the OpenAI API key
openai_api_key = os.getenv("MY_OPENAI_KEY")
TMDB_API_KEY = os.getenv("TMDB_API_KEY")

In [7]:
logging.basicConfig(level=logging.INFO)

def search_movie(keywords):
    base_url = "https://api.themoviedb.org/3/search/movie"
    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {TMDB_API_KEY}"
        }

    results = []
    for keyword in keywords:
        params = {
            "query": keyword,
            "include_adult": "false",
            "language": "en-US",
            "page": 1
        }

        response = requests.get(base_url, headers=headers, params=params)

        if response.status_code == 200:
            logging.info(
                'TMDB API request successful for keyword: %s', keyword)
            results.extend(response.json()['results'])
        else:
            logging.error('TMDB API request failed with status code: %s for keyword: %s. Details: %s',
                response.status_code, keyword, response.text)

    results = process_movies(results)

    return results

In [9]:
def process_movies(movies):
    current_year = datetime.now().year

    # Filter for movies from the last 3 years
    recent_movies = [movie for movie in movies if movie['release_date'] and int(movie['release_date'][:4]) >= current_year - 10]

    # Remove duplicates
    unique_movies = {movie['id']: movie for movie in recent_movies}.values()

    # Sort by popularity
    sorted_movies = sorted(unique_movies, key=lambda m: m['popularity'], reverse=True)

    # Return the top 21, or less if there aren't that many
    return sorted_movies[:21]

In [11]:
def converse_with_openai(prompt):
    message = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
    ]

    response = ChatOpenAI.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=message,
    )

    logging.info('OpenAI API request successful')

    # Return the assistant's reply
    return response['choices'][0]['message']['content']

In [13]:
def generate_query(text):
    movie_props = {f"movie_{i}": {"type": "string", "description": f"Movie {i}"} for i in range(1, 11)}

    message = [
        {"role": "system",
         "content": "You are a helpful assistant. Get each movie in the text. Movies should just be strings of characters in lowercase, no number or date or parenthesis."},
        {"role": "user", "content": text},
    ]

    response = ChatOpenAI.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=message,
        functions=[{
            "name": "get_movies",
            "description": "Returns 4 movies as the base search",
            "parameters": {
                "type": "object",
                "properties": movie_props,
                "required": list(movie_props.keys()),
            },
        }],
        function_call={"name": "get_movies"},
    )

    logging.info('OpenAI API request successful')

    message = response["choices"][0]["message"]
    movies = []

    if message.get("function_call"):
        arguments = json.loads(message["function_call"]["arguments"])
        print(arguments)
        movies = [arguments[f"movie_{i}"] for i in range(1, 11)]

    return movies

In [14]:
def get_movie_suggestion(user_preference):
    # Here we generate a prompt to ask GPT-3 about movie
    # suggestions based on user preferences
    prompt = f"""I am looking for a movie. My preferences are:
                    {user_preference}. Can you suggest something? (10 movies)"""
    logging.info('Getting movie suggestion from OpenAI with prompt: %s', prompt)

    movie_gpt = converse_with_openai(prompt)
    logging.info('Received movie suggestion from OpenAI: %s', movie_gpt)

    movie_query = generate_query(movie_gpt)

    logging.info('Received movies keywords from OpenAI: %s', movie_query)

    return search_movie(movie_query)